In [1]:
#!pip install -U jiwer

In [2]:
#!pip install evaluate

In [3]:
import os
from pydub import AudioSegment
from jiwer import wer #Para la métrica
import re
from statistics import mean, stdev
import PyPDF2

import string 
import nltk
nltk.download('punkt')
nltk.download('stopwords')
from nltk.tokenize import word_tokenize
import num2words
from autocorrect import Speller #Para los errores ortográficos
spell = Speller(lang='es')
from jiwer import wer #Para la métrica
import time # Para medir los tiempos

[nltk_data] Downloading package punkt to /home/joheras/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/joheras/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
# Para cargar el texto original si el audio es del COSER
def originalCOREC(nombre):
    with open('../COREC/'+nombre+'.pdf', 'rb') as file:
        pdf = PyPDF2.PdfReader(file)
    
        texto = ''
    
        for pagina in range(len(pdf.pages)):
            pagina_texto = pdf.pages[pagina].extract_text()
            texto = texto + pagina_texto
            
    return texto

def limpiarCOREC(original):
    
    original = original.replace('//', '. ').replace('/ ', ', ').replace('/', ', ') # Cambiamos las barras por puntos y comas
    
    original = re.sub(' \[\d\d:\d\d\]', '', original) # Quitamos las marcas de tiempo
    original = re.sub('\(\d:\d\d\)', '', original)
    original = re.sub('\(\d\d:\d\d\)', '', original)
    original = re.sub('\[.*\]{1}', '', original)
    original = re.sub('\(.*\)', '', original)
    original = re.sub('[(][^)]*[)]', '', original)
    original = re.sub('[\[][^\]]*[\]]', '', original)
    
    #original = original.replace('[', '').replace(']', '')

    
    encuestadores = ['E:', 'e:', 'E1:', 'E2:', 'E3:', 'E4:', 'E =', 'E=', 'E1 =', 'E1=', 'E2 =', 'E2=', 'E3 =', 'E3=', 'E4 =', 'E4=', 'E.1', 'E.2', 'E.3', 'E.4', 'E1.TL', 'E2.TL', 'E3.TL', 'E4.TL'] # Quitamos la marca del encuestador
    for encuest in encuestadores:
        original = original.replace(encuest, ' ')
        
    informantes = ['I:', 'I1:', 'I2:', 'I3:', 'I4:', 'NP:', 'NP', 'Mote: ', 'XXX', 'I =', 'I=', 'I1 =', 'I1=', 'I2 =', 'I2=', 'I3 =', 'I3=', 'I4 =', 'I4=', 'I.1', 'I.2', 'I.3', 'I.3', 'I1.TL', 'I2.TL', 'I3.TL', 'I4.TL'] # Quitamos la marca del informante
    for inf in informantes:
        original = original.replace(inf, ' ')
        
        
    original = original.replace('/n', ' ') # Quitamos los saltos de línea
    original = original.replace('\n', ' ')       
        
    original = re.sub(' +', ' ', original)
 
    return original

def originalCOSER(nombre):
    
    
    with open(nombre, 'r', encoding = 'ISO-8859-1') as f:
        original = f.read()
    f.close()
    
    
    # Sacamos los datos del principio:
    copia = []
    copia.append(original)
    
    datos = {}
    for c in range(len(copia)):
        copia[c] = copia[c].split('\n')  # Separamos los txt por párrafos
        for i in range(len(copia[c])):
            copia[c][i] = copia[c][i].split('\t')
    copia = copia[0]
    
    cont = 0
    while copia[cont][0]!= 'Informantes:': # Nos guardamos la información hasta informantes
        datos[copia[cont][0][:-1]] = copia[cont][1]
        cont = cont + 1
    datos[copia[cont][0][:-1]] = [copia[cont][1]]
    a = cont
    cont = cont +1
    while 'Encuest' not in copia[cont][0]:
        datos[copia[a][0][:-1]].append(copia[cont][0])
        cont = cont+1
        
    while copia[cont][0] != 'Temas:':
        datos[copia[cont][0][:-1]] = copia[cont][1]
        cont = cont + 1
    datos[copia[cont][0][:-1]] = []
    datos[copia[cont][0][:-1]].append(copia[cont][1])
    b = cont
    cont = cont+1
    while copia[cont]!=['']:
        datos[copia[b][0][:-1]].append(copia[cont][0])
        cont = cont+1
        
    cont = cont+1

    texto = ''
    
    while copia[cont]!=['']:
        texto = texto + copia[cont][0]
        cont = cont + 1
    #while copia[cont]!=['']:
    #    if "HS:" in copia[cont][0]:
    #        copia[cont][0] = 'CR, '+copia[cont][0]
   #     datos[original[-1]].append(copia[cont])
   #     cont = cont+1

    #texto = ''
    #for i in range(len(datos)):
    #    texto = texto + datos[i][0]
    #return datos, texto
    return datos, texto



def limpiarCOSER(original):
    
    original = re.sub(' \[\d\d:\d\d\]', '', original) # Quitamos las marcas de tiempo
    
    original = re.sub('\|T\d\|', '', original) # Quitamos las marcas del tema que se trata
    original = re.sub('\|T\d\d\|', '', original)
    original = re.sub('\|\d\|', '', original) 
    original = re.sub('[(][^)]*[)]', '', original)
    original = re.sub('[\[][^\]]*[\]]', '', original)
    
    encuestadores = ['E: ', 'E1: ', 'E2: ', 'E3: ', 'E4: '] # Quitamos la marca del encuestador
    for encuest in encuestadores:
        original = original.replace(encuest, ' ')
        
    informantes = ['I: ', 'I1: ', 'I2: ', 'I3: ', 'I4: ', 'NP: ', 'NP', 'Mote: ', 'XXX'] # Quitamos la marca del informante
    for inf in informantes:
        original = original.replace(inf, ' ')
    
    solapamientos = ['HS:E1', 'HS:E2', 'HS:E3', 'HS:E4', 'HS:E', 'HS:I1', 'HS:I2', 'HS:I3',
                     'HS:I', 'HS:', 'HCruz:', 'HSim-O:', 'V-Sml', 'V-Ljn', 'V-Mrm', 'V-Otr', 'V-Tml']
    for solap in solapamientos:
        original = original.replace(solap, '')
        
    pronunciacion = ['P-Enf:', 'P-Rel:', 'P-Ssr:', 'P-Slb:', 'P-Otr:']
    for pronu in pronunciacion:
        original = original.replace(pronu, '')
        
    emisiones = ['Asent', 'RISAS', 'RISA', 'Rndo: ', 'Llndo: ', 'EXCL', 'TOS', 'CARRASP', 'CHASQ', 'ONOM',
                 'RESPIR', 'OTRAS-EM']
    for emi in emisiones:
        original = original.replace(emi, ' ')
        
    pausas = ['PS', 'PS: ', 'SLNC']
    for pausa in pausas:
        original = original.replace(pausa, '')
        
    emisionesExternas = ['R-Inf', 'R-Vhc', 'R-Anm', 'R-Vcs', 'R-Cas', 'R-Prt', 'R-Glp', 'R-Mus', 'R-Cmp', 
                         'R-Vnt', 'R_Tlf', 'R-Ppl', 'R-Grb', 'R-Mcr', 'R-Ind']
    for emis in emisionesExternas:
        original = original.replace(emis, '')
        
    gestos = ['G-Mst', 'G-Imt', 'G-Gnr', 'G-Otr']
    for gesto in gestos:
        original = original.replace(gesto, '')
        
    inteligibilidad = ['A-Inn', 'A-PIn:', 'A-Nul', 'A-Pau', 'A-Crt', 'A-Err', 'A-InfErr', 'AT']
    for intel in inteligibilidad:
        original = original.replace(intel, '')
        
    literatura = ['LT-Rlt:', 'LT-Rct:', 'LT-Cnt:', 'LT-Rfn:', 'LT-Otr:', 'L-Otra:']
    for lit in literatura:
        original = original.replace(lit, '')
 
    return original





def normalizar(texto):
    #print(texto)
    tokens=word_tokenize(texto) #Separa el texto por palabras
    #print(tokens)
    texto = [w.lower() for w in tokens] #Pasamos las palabras a minúsculas
    #print(texto)
    texto = [texto[i].replace(' apdo.', ' apartado').replace(' art. ',' articulo ').replace(' atte.',' atentamente').replace(' avda.',' avenida').replace(' cap.',' capítulo').replace(' cía.',' compañía').replace(' coord.',' coordinador').replace(' d.',' don').replace(' dña.',' doña').replace(' dcho.',' derecho').replace(' dcha.',' derecha').replace(' depto.',' departamento').replace(' dr.',' doctor').replace(' dra.',' doctora').replace(' etc.',' etcétera').replace(' fdo.',' firmado').replace(' izqdo.',' izquierdo').replace(' izqda.',' izquierda').replace(' max.',' máximo').replace(' min.',' mínimo').replace(' núm.',' número').replace(' pág.',' página').replace(' ej.',' ejemplo').replace(' prov.',' provincia').replace(' sr.',' señor').replace(' sra.',' señora').replace(' srta.',' señorita').replace(' tfno.',' teléfono') for i in range(len(texto))] #quitamos las abreviaciones
    for i in range(len(texto)):
        if texto[i].isdigit() and (texto[i] not in string.punctuation):
            texto[i]=num2words.num2words(texto[i], lang='es', ordinal=False)
    tokens=[w.lower() for w in texto] #Pasamos las palabras a minúsculas por si había números
    re_punc = re.compile('[%s]' % re.escape(string.punctuation))
    stripped = [re_punc.sub('',w) for w in tokens]
    stripped=[stripped[i].replace('¿', '').replace('¡','').replace("'",'') for i in range(len(tokens))] #quita los símbolos de puntuación que string.punctuation no tiene en cuenta
    words = [word for word in stripped if word.isalpha()] #Elimina los signos de puntuación
    return words

# Para quedarnos con el texto normalizado y sin errores ortográficos
def correcto(texto):
    result = normalizar(texto)
    for word in result:
        word = spell(word)
        
    result_norm = ""
    for i in range(len(result)):
        result_norm = result_norm + ' ' + result[i]
    result = result_norm
    return result


In [5]:
datos, original = originalCOSER('../COSER-4117-01.TXT') 
original = limpiarCOSER(original)
original = correcto(original)

In [6]:
with open('../audios_large-v3/COSER-4117-01.txt') as f:
    predicted = f.read()

In [7]:
from evaluate import load
wer = load("wer")

2024-03-21 16:40:04.871927: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT



===================================BUG REPORT===================================
Welcome to bitsandbytes. For bug reports, please submit your error trace to: https://github.com/TimDettmers/bitsandbytes/issues


In [8]:
wer.compute(predictions=[predicted],references=[original])

0.13326621783174983

In [9]:
from jiwer import compute_measures

In [10]:
measures = compute_measures([original], [predicted])

In [11]:
measures['deletions'],measures['substitutions'],measures['insertions']

(671, 1099, 348)

In [12]:
deleted = []
dele = ''
for x in measures['ops'][0]:
    if(x.type=='delete'):
        deleted.append(str(measures['truth'][0][x.ref_start_idx:x.ref_end_idx]))
for elem in deleted:
    dele = dele + elem +'\n'
with open('delete-COSER-4117-01', 'a', encoding = 'utf-8') as f:
    f.write(dele)
f.close()
    

In [13]:
substituted = []
subs = ''
for x in measures['ops'][0]:
    if(x.type=='substitute'):
        substituted.append(str(measures['truth'][0][x.ref_start_idx:x.ref_end_idx]) + " -->" + str(measures['hypothesis'][0][x.hyp_start_idx:x.hyp_end_idx]))
for elem in substituted:
    subs = subs + elem + '\n'
with open('substitute-COSER-4117-01', 'a', encoding='utf-8') as f:
    f.write(subs)
f.close()

In [14]:
inserted = []
ins = ''
for x in measures['ops'][0]:
    if(x.type=='insert'):
        inserted.append(measures['hypothesis'][0][x.hyp_start_idx:x.hyp_end_idx])
for elem in inserted:
    ins = ins + str(elem) + '\n'
with open('insert-COSER-4117-01', 'a', encoding='utf-8') as f:
    f.write(ins)
f.close()